In [1]:
from __future__ import print_function
from unicodecsv import DictReader
import pandas as pd
from paths import DATASETS_DIR, DUMPS_DIR
from pprint import pprint
from data import generate_dump
import os
from collections import OrderedDict
# import mysql.connector
from sqlalchemy import create_engine
# import pyodbc
from helpers import freq_iso_to_pandas
from pydatajson.helpers import parse_repeating_time_interval_to_days
from generate_dumps import save_to_dta
import numpy as np

In [2]:
dump_path = os.path.join(DUMPS_DIR, "series-tiempo.{}")

## Genero dataframe con el dump

In [3]:
%time df = generate_dump()

CPU times: user 58.4 s, sys: 2.65 s, total: 1min 1s
Wall time: 1min 1s


In [4]:
len(df)

791812

In [11]:
len(df[df.valor.isnull()]) / float(len(df))

0.0008322682656994337

In [5]:
len(df)

842481

In [33]:
df_sorted = df.sort_values(["catalog_id", "dataset_id", "distribucion_id", "serie_id", "indice_tiempo"], ascending=True)
len(df_sorted)

842481

In [36]:
def remove_leading_nans(df, serie_id):
    df_serie = df[df.serie_id == serie_id]
    first_positive = df_serie[df_serie.valor.notnull()].index[0]
    df[df.serie_id == serie_id] = df_serie.loc[first_positive:]

In [37]:
%%time
for serie_id in df_sorted.serie_id.unique():
    remove_leading_nans(df_sorted, serie_id)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
len(df_sorted)

In [29]:
df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"] = df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"].loc[first:]

In [23]:
first = df[df.serie_id == "149.1_TL_INDIIOS_OCTU_0_21"][df.valor.notnull()].index[0]

/Users/abenassi/anaconda/envs/series-tiempo/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


## Genero resumen de series

In [41]:
drop_cols = ["indice_tiempo", "valor"]
index_cols = ["catalog_id", "dataset_id", "distribucion_id", "serie_id"]
series_cols = df.columns.drop(drop_cols)
series_group = df.groupby(index_cols)

In [42]:
df_series = df.drop(drop_cols, axis=1).drop_duplicates().set_index(index_cols)

In [43]:
def _is_series_updated(row):
    index_freq = row["indice_tiempo_frecuencia"]
    period_days = parse_repeating_time_interval_to_days(index_freq)
    periods_tolerance = {
        "R/P1Y": 2,
        "R/P6M": 2,
        "R/P3M": 2,
        "R/P1M": 3,
        "R/P1D": 14
    }
    days_tolerance = periods_tolerance.get(index_freq, 2) * period_days
    return row["serie_dias_no_cubiertos"] < days_tolerance    

In [44]:
def _get_serie_valor_anterior_anio(series_dataframe):
    series = pd.Series(list(series_dataframe.valor), list(
        series_dataframe.indice_tiempo)).sort_index()
    return series.get(series.index[-1] - pd.DateOffset(years=1))

In [45]:
# CALCULA INDICADORES resumen sobre las series
# rango temporal de la serie
df_series["serie_indice_inicio"] = series_group["indice_tiempo"].min()
df_series["serie_indice_final"] = series_group["indice_tiempo"].max()
df_series["serie_valores_cant"] = series_group["indice_tiempo"].count()

# estado de actualización de los datos
# calcula días que pasaron por encima de período cubierto por datos
df_series["serie_dias_no_cubiertos"] = df_series.apply(
    lambda x: (pd.datetime.now() - pd.to_datetime(x[
        "serie_indice_final"]).to_period(
        freq_iso_to_pandas(x[
            "indice_tiempo_frecuencia"],
            how="end")).to_timestamp(how="end")).days,
    axis=1)
# si pasaron 2 períodos no cubiertos por datos, serie está desactualizada
df_series["serie_actualizada"] = df_series.apply(
    _is_series_updated, axis=1)

# valores representativos nominales
df_series["serie_valor_ultimo"] = series_group.apply(
    lambda x: x.loc[x.indice_tiempo.argmax(), "valor"])
df_series["serie_valor_anterior"] = series_group.apply(
    lambda x: pd.Series(list(x.valor), list(
        x.indice_tiempo)).sort_index()[-2]
)
df_series["serie_valor_anterior_anio"] = series_group.apply(
    _get_serie_valor_anterior_anio)

# valores representativos en variación porcentual
df_series["serie_var_pct_anterior"] = df_series[
    "serie_valor_ultimo"] / df_series["serie_valor_anterior"] - 1
df_series["serie_var_pct_anterior_anio"] = df_series[
    "serie_valor_ultimo"] / df_series["serie_valor_anterior_anio"] - 1

In [50]:
df_series[~(df_series.serie_actualizada) & (df_series.indice_tiempo_frecuencia == "R/P1M")].sort_values(
    "serie_dias_no_cubiertos", ascending=True)

indice_tiempo_frecuencia  \
catalog_id dataset_id distribucion_id serie_id                                           
sspm       56         56.2            56.2_SPSMD_2012_M_26                       R/P1M   
                                      56.2_SPRSMD_2012_M_33                      R/P1M   
                                      56.2_NGSMD_2012_M_26                       R/P1M   
                                      56.2_SPMD_2012_M_22                        R/P1M   
                                      56.2_SPNRMD_2012_M_32                      R/P1M   
                                      56.2_SPRMD_2012_M_29                       R/P1M   
                                      56.2_SPNRSMD_2012_M_36                     R/P1M   
                                      56.2_NGMD_2012_M_22                        R/P1M   
           115        115.3           115.3_TCRM_0_M_29                          R/P1M   
           125        125.2           125.2_HAFB_1993_0_29                       R/P1M   
                                      125.2_TAP_1993_0_25                        R/P1M   
                                      125.2_CC_1993_0_18                         R/P1M   
                                      125.2_PP_1993_0_21                         R/P1M   
                                      125.2_RCM_1993_0_22                        R/P1M   
                                      125.2_HMAF_1993_0_40                       R/P1M   
                                      125.2_VAM_1993_0_33                        R/P1M   
                                      125.2_MAA_1993_0_24                        R/P1M   
                                      125.2_GR_1993_0_28                         R/P1M   
                                      125.2_MUEIMA_1993_0_58                     R/P1M   
                                      125.2_SPE_1993_0_34                        R/P1M   
                                      125.2_CR_1993_0_23                         R/P1M   
                                      125.2_OMUG_1993_0_29                       R/P1M   
                                      125.2_EM_1993_0_18                         R/P1M   
                                      125.2_GVL_1993_0_21                        R/P1M   
           126        126.2           126.2_SPEA_1993_0_37                       R/P1M   
           125        125.2           125.2_AM_1993_0_21                         R/P1M   
                                      125.2_OMT_1993_0_25                        R/P1M   
                                      125.2_CC_1993_0_13                         R/P1M   
                                      125.2_ETV_1993_0_26                        R/P1M   
                                      125.2_A_1993_0_8                           R/P1M   
...                                                                                ...   
           122        122.2           122.2_AP_1999_0_11                         R/P1M   
                                      122.2_PP_1999_0_22                         R/P1M   
                                      122.2_EF_1999_0_17                         R/P1M   
                                      122.2_QS_1999_0_16                         R/P1M   
                                      122.2_TM_1999_0_10                         R/P1M   
                                      122.2_CV_1999_0_12                         R/P1M   
                                      122.2_CA_1999_0_9                          R/P1M   
                                      122.2_HT_1999_0_12                         R/P1M   
                                      122.2_PF_1999_0_10                         R/P1M   
           100        100.3           100.3_I2NG_2003_M_22                       R/P1M   
                                      100.3_I2I_2003_M_21                        R/P1M   
                                      100.3_I2TC_2003_M_34                       R/P1M   
            

In [20]:
df_series.to_excel("series-tiempo-resumen.xlsx", "resumen", index=True, index_label=True, merge_cells=False)

In [29]:
df_stata = df_series.copy()

In [35]:
for col in df_series.columns:
    
    # limita el largo de los campos de texto
    if df_stata[col].dtype.name == "object":
        df_stata[col] = df_stata[col].str[:244]
    
    # elimina los valores infinitos de los tipos decimales
    elif "float" in df_stata[col].dtype.name:
        df_stata[col] = df_stata[col].apply(lambda x: np.nan if np.isinf(x) else x)

In [36]:
df_stata.to_stata("series-tiempo-resumen.dta", write_index=False)

In [34]:
df_series.var_pct_anterior.max()

inf

In [ ]:
df_series.to_sql()

## Genero distribuciones de distintos formatos

In [ ]:
%%time
# CSV
path = dump_path.format("csv")
df.to_csv(path, encoding="utf-8", sep=",", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# EXCEL
path = dump_path.format("xlsx")
writer = pd.ExcelWriter(path, engine='xlsxwriter')
df.to_excel(writer, "data", merge_cells=False, encoding="utf-8", index=False)
writer.save()
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQLITE
path = dump_path.format("db")
engine = create_engine('sqlite:///{}'.format(path), echo=True)
df.to_sql("series_tiempo", engine, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# ACCESS
# set up some constants
DRV = '{Microsoft Access Driver (*.mdb)}'; PWD = 'pw'

# connect to db
path = dump_path.format("mdb")
con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV, path, PWD))
cur = con.cursor()

df.to_sql("series_tiempo", con, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQL
# la gracia acá es hacer bien las tablas con sus índices y pensar si MySQL, Postgresql o SQLite
path = dump_path.format("sql")
df.to_sql(path, "data", merge_cells=False, encoding="utf-8", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [22]:
%%time 
# DTA
# hay que ver cómo manejar el límite de caracteres
path = dump_path.format("dta")
df_stata = df.copy()
for col in df_stata.columns:
    if df_stata[col].dtype.name == "object":
        # trunca campos de texto a 244 caracteres aceptados por STATA
        df_stata[col] = df_stata[col].str[:244]
df_stata.to_stata(path, write_index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

1097MB
CPU times: user 20.1 s, sys: 5.21 s, total: 25.3 s
Wall time: 25.6 s


In [ ]:
df.head()